In [ ]:
%pip install azure-ai-ml

# To use the latest version of langchain, please uncomment the line right below
# and remove the `langchain` extra. Otherwise, the `langchain` extra will be used instead.
# %pip install langchain
%pip install -U 'azureml-rag[pinecone,langchain]>=0.2.11'

# If using hugging_face embeddings add `hugging_face` extra, e.g. `azureml-rag[pinecone,hugging_face]`

# Create a Pinecone-based Vector Index from Data in S3 Using AzureML Data Import

We'll walk through setting up an AzureML Pipeline which imports data from an S3 bucket, processes the data into chunks, embeds the chunks and creates a LangChain-compatible Pinecone MLIndex.

Note: Support for [namespaces](https://docs.pinecone.io/docs/namespaces) is currently not available. We will be actively working on it and are excited to bring this feature to you!

## Get client for AzureML Workspace

The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

If you don't have a Workspace and want to create and Index locally see [here to create one](https://learn.microsoft.com/azure/machine-learning/quickstart-create-resources?view=azureml-api-2).

Enter your Workspace details below, running this still will write a `workspace.json` file to the current folder.

In [ ]:
%%writefile workspace.json
{
    "subscription_id": "<subscription_id>",
    "resource_group": "<resource_group_name>",
    "workspace_name": "<workspace_name>"
}

`MLClient` is how you interact with AzureML

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path="workspace.json")
except Exception as ex:
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace details."
    ) from ex
    # ml_client = MLClient(
    #     credential=credential,
    #     subscription_id="",
    #     resource_group_name="",
    #     workspace_name=""
    # )

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)

print(ml_client)

## Which Embeddings Model to use?

There are currently two supported Embedding options: OpenAI's `text-embedding-ada-002` embedding model or HuggingFace embedding models. Here are some factors that might influence your decision:

### OpenAI

OpenAI has [great documentation](https://platform.openai.com/docs/guides/embeddings) on their Embeddings model `text-embedding-ada-002`, it can handle up to 8191 tokens and can be accessed using [Azure OpenAI](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models#embeddings-models) or OpenAI directly.
If you have an existing Azure OpenAI Instance you can connect it to AzureML, if you don't AzureML provisions a default one for you called `Default_AzureOpenAI`.
The main limitation when using `text-embedding-ada-002` is cost/quota available for the model. Otherwise it provides high quality embeddings across a wide array of text domains while being simple to use.

### HuggingFace

HuggingFace hosts many different models capable of embedding text into single-dimensional vectors. The [MTEB Leaderboard](https://huggingface.co/spaces/mteb/leaderboard) ranks the performance of embeddings models on a few axis, not all models ranked can be run locally (e.g. `text-embedding-ada-002` is on the list), though many can and there is a range of larger and smaller models. When embedding with HuggingFace the model is loaded locally for inference, this will potentially impact your choice of compute resources.

**NOTE:** The default PromptFlow Runtime does not come with HuggingFace model dependencies installed, Indexes created using HuggingFace embeddings will not work in PromptFlow by default. **Pick OpenAI if you want to use PromptFlow**

### Run the cells under _either_ heading (OpenAI or HuggingFace) to use the respective embedding model

#### OpenAI

We can use the automatically created `Default_AzureOpenAI` connection.

If you would rather use an existing Azure OpenAI connection then change `aoai_connection_name` below.
If you would rather use an existing Azure OpenAI resource, but don't have a connection created, modify `aoai_connection_name` and the details under the `# Create New Connection` code comment, or navigate the PromptFlow section in your AzureML Workspace and use the Connections create UI flow.

In [ ]:
aoai_connection_name = "Default_AzureOpenAI"
aoai_connection_id = None

Here we get the connection from the Workspace and save its `id` so we can reference it later.

In [ ]:
from azureml.rag.utils.connections import (
    get_connection_by_name_v2,
    create_connection_v2,
)

try:
    aoai_connection = get_connection_by_name_v2(ws, aoai_connection_name)
except Exception as ex:
    # Create New Connection
    # Modify the details below to match the `Endpoint` and API key of your AOAI resource, these details can be found in Azure Portal
    raise RuntimeError(
        "Have you entered your AOAI resource details below? If so, delete me!"
    )
    aoai_connection = create_connection_v2(
        workspace=ws,
        name=aoai_connection,
        category="AzureOpenAI",
        # 'Endpoint' from Azure OpenAI resource overview
        target="https://<endpoint_name>.openai.azure.com/",
        auth_type="ApiKey",
        credentials={
            # Either `Key` from the `Keys and Endpoint` tab of your Azure OpenAI resource, will be stored in your Workspace associated Azure Key Vault.
            "key": "<api-key>"
        },
        metadata={"ApiType": "azure", "ApiVersion": "2023-05-15"},
    )

aoai_connection_id = aoai_connection["id"]

Now that your Workspace has a connection to Azure OpenAI we will make sure the `text-embedding-ada-002` model has been deployed ready for inference. This cell will fail if there is not deployment for the embeddings model, [follow these instructions](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#deploy-a-model) to deploy a model with Azure OpenAI.

In [ ]:
from azureml.rag.utils.deployment import infer_deployment

aoai_embedding_model_name = "text-embedding-ada-002"
try:
    aoai_embedding_deployment_name = infer_deployment(
        aoai_connection, aoai_embedding_model_name
    )
    print(
        f"Deployment name in AOAI workspace for model '{aoai_embedding_model_name}' is '{aoai_embedding_deployment_name}'"
    )
except Exception as e:
    print(
        f"Deployment name in AOAI workspace for model '{aoai_embedding_model_name}' is not found."
    )
    print(
        f"Please create a deployment for this model by following the deploy instructions on the resource page for '{aoai_connection['properties']['target']}' in Azure Portal."
    )

Finally we will combine the deployment and model information into a uri form which the AzureML embeddings components expect as input.

In [ ]:
embeddings_model_uri = f"azure_open_ai://deployment/{aoai_embedding_deployment_name}/model/{aoai_embedding_model_name}"

#### HuggingFace

AzureML's default model from HuggingFace is `all-mpnet-base-v2`, it can be run by most laptops. Any `sentence-transformer` models should be supported, you can learn more about `sentence-transformers` [here](https://huggingface.co/sentence-transformers).

In [ ]:
embeddings_model_uri = "hugging_face://model/sentence-transformers/all-mpnet-base-v2"

## Prepare Pinecone Index

If you would rather use an existing Pinecone index from a Pinecone project connection then change `pinecone_connection_name` below.
If you would rather use an existing Pinecone index, but don't have a connection to the Pinecone project created, modify `pinecone_connection_name` and the details under the `# Create New Connection` code comment.

Note: Only Custom Connections are supported right now for connecting to a Pinecone project (where your index lives).

If creating the connection through the Create Connections UI flow, then please specify the Pinecone project
- API key (must use `api_key` as the key for the key-value pair)
- ID (must use `project_id` as the key for the key-value pair)
- environment (must use `environment` as the key for the key-value pair)

as key-value pairs in the Custom Connection where the API key key-value pair must be a secret. For example:

![Create Pinecone Connection UI](../../media/pinecone_connection_ui.png)


You can also modify the details under the `# Create New Connection` code comment to create a connection via the SDK!

In [ ]:
pinecone_connection_name = "my_pinecone_project_connection"
pinecone_connection_id = None

In [ ]:
from azureml.rag.utils.connections import (
    get_connection_by_name_v2,
    create_connection_v2,
)

try:
    pinecone_connection = get_connection_by_name_v2(ws, pinecone_connection_name)
except Exception as ex:
    # Create New Connection
    # Modify the details below to match the details of the Pinecone project where your index lives, more details here: https://docs.pinecone.io/docs/projects
    raise RuntimeError(
        "Have you entered your Pinecone project details below? If so, delete me!"
    )
    pinecone_connection = create_connection_v2(
        workspace=ws,
        name=pinecone_connection_name,
        category="CustomKeys",
        target="_",
        auth_type="CustomKeys",
        credentials={
            "keys": {
                # https://docs.pinecone.io/docs/projects#api-keys
                "api_key": "<api-key>"
            }
        },
        metadata={
            # https://docs.pinecone.io/docs/projects#project-environment
            "environment": "<environment>",
            # https://docs.pinecone.io/docs/projects#project-id
            "project_id": "<project_id>",
        },
    )

pinecone_connection_id = pinecone_connection["id"]

## Setup Pipeline to process data into Index

### Setup S3 Data Import

You will need to modify the `s3.yaml` file in this directory to include the information specific to your s3 bucket. There are other supported Data Import source [outline here](../../../../../../cli/assets/data/README.md).

In [ ]:
from azure.ai.ml import load_workspace_connection

s3_connection_name = "my_s3_bucket_connection"

# Modify `s3.yaml` to include your S3 bucket details
ws_connection = load_workspace_connection(source="./s3.yaml")
ml_client.connections.create_or_update(workspace_connection=ws_connection)

Now we can create a DataImport job that produces a new Data Asset containing data from the S3 bucket.

In [ ]:
from azure.ai.ml.entities import DataImport
from azure.ai.ml.data_transfer import import_data
from azure.ai.ml.data_transfer import FileSystem

# `${{name}}` will be replaced by the Import Job's id, which is a uuid.
s3_import_path = "azureml://datastores/workspaceblobstore/paths/s3_import/${{name}}"
import_asset_name = "s3_import_test"

data_import = DataImport(
    name=import_asset_name,
    source=FileSystem(connection=f"azureml:{s3_connection_name}", path="*"),
    path=s3_import_path,
)
ml_client.data.import_data(data_import=data_import)

### Setup Pinecone MLIndex creation pipeline

AzureML [Pipelines](https://learn.microsoft.com/azure/machine-learning/concept-ml-pipelines?view=azureml-api-2) connect together multiple [Components](https://learn.microsoft.com/azure/machine-learning/concept-component?view=azureml-api-2). Each Component defines inputs, code that consumes the inputs and outputs produced from the code. Pipelines themselves can have inputs, and outputs produced by connecting together individual sub Components.
To process your data for embedding and indexing we will chain together multiple components each performing their own step of the workflow.

The Components are published to a [Registry](https://learn.microsoft.com/azure/machine-learning/how-to-manage-registries?view=azureml-api-2&tabs=cli), `azureml`, which should have access to by default, it can be accessed from any Workspace.
In the below cell we get the Component Definitions from the `azureml` registry.

In [ ]:
ml_registry = MLClient(credential=credential, registry_name="azureml")

# Walks input folder according to provided glob pattern (all files by default: '**/*') and attempts to open them, extract text chunks and further chunk if necessary to fir within provided `chunk_size`.
crack_and_chunk_component = ml_registry.components.get(
    "llm_rag_crack_and_chunk", label="latest"
)
# Reads input folder of files containing chunks and their metadata as batches, in parallel, and generates embeddings for each chunk. Output format is produced and loaded by `azureml.rag.embeddings.EmbeddingContainer`.
generate_embeddings_component = ml_registry.components.get(
    "llm_rag_generate_embeddings", label="latest"
)
# Reads an input folder produced by `azureml.rag.embeddings.EmbeddingsContainer.save()` and pushes all embeddings (including metadata) into a Pinecone index. Writes an MLIndex yaml detailing the index and embeddings model information.
update_pinecone_index_component = ml_registry.components.get(
    "llm_rag_update_pinecone_index", label="latest"
)
# Takes a uri to a storage location where an MLIndex yaml is stored and registers it as an MLIndex Data asset in the AzureML Workspace.
register_mlindex_component = ml_registry.components.get(
    "llm_rag_register_mlindex_asset", label="latest"
)

Each Component has documentation which provides an overall description of the Components purpose and each of the inputs/outputs.
For example we can see understand what `crack_and_chunk` does by inspecting the Component definition.

In [ ]:
print(crack_and_chunk_component)

Below a Pipeline is built by defining a python function which chains together the above components inputs and outputs. Arguments to the function are inputs to the Pipeline itself and the return value is a dictionary defining the outputs of the Pipeline.

In [ ]:
from azure.ai.ml import Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities._job.pipeline._io import PipelineInput
from typing import Optional


def use_automatic_compute(component, instance_count=1, instance_type="Standard_E8s_v3"):
    """Configure input `component` to use automatic compute with `instance_count` and `instance_type`.

    This avoids the need to provision a compute cluster to run the component.
    """
    component.set_resources(
        instance_count=instance_count,
        instance_type=instance_type,
        properties={"compute_specification": {"automatic": True}},
    )
    return component


def optional_pipeline_input_provided(input: Optional[PipelineInput]):
    """Checks if optional pipeline inputs are provided."""
    return input is not None and input._data is not None


# If you have an existing compute cluster you want to use instead of automatic compute, uncomment the following line, replace `dedicated_cpu_compute` with the name of your cluster.
# Also comment out the `component.set_resources` line in `use_automatic_compute` above and the `default_compute='serverless'` line below.


# @pipeline(compute=dedicated_cpu_compute)
@pipeline(default_compute="serverless")
def s3_into_pinecone(
    input_data: Input,
    embeddings_model: str,
    pinecone_config: str,
    pinecone_connection_id: str,
    asset_name: str,
    chunk_size: int = 1024,
    data_source_glob: str = None,
    data_source_url: str = None,
    document_path_replacement_regex: str = None,
    aoai_connection_id: str = None,
    embeddings_container: Input = None,
):
    """Pipeline to generate embeddings for a `input_data` source and push them into a Pinecone index."""

    crack_and_chunk = crack_and_chunk_component(
        input_data=input_data,
        input_glob=data_source_glob,
        chunk_size=chunk_size,
        data_source_url=data_source_url,
        document_path_replacement_regex=document_path_replacement_regex,
    )
    use_automatic_compute(crack_and_chunk)

    generate_embeddings = generate_embeddings_component(
        chunks_source=crack_and_chunk.outputs.output_chunks,
        embeddings_container=embeddings_container,
        embeddings_model=embeddings_model,
    )
    use_automatic_compute(generate_embeddings)
    if optional_pipeline_input_provided(aoai_connection_id):
        generate_embeddings.environment_variables[
            "AZUREML_WORKSPACE_CONNECTION_ID_AOAI"
        ] = aoai_connection_id
    if optional_pipeline_input_provided(embeddings_container):
        # If provided, `embeddings_container` is expected to be a URI to folder, the folder can be empty.
        # Each sub-folder is generated by a `create_embeddings_component` run and can be reused for subsequent embeddings runs.
        generate_embeddings.outputs.embeddings = Output(
            type="uri_folder", path=f"{embeddings_container.path}/{{name}}"
        )

    # `update_pinecone_index` takes the Embedded data produced by `generate_embeddings` and pushes it into a Pinecone index.
    update_pinecone_index = update_pinecone_index_component(
        embeddings=generate_embeddings.outputs.embeddings,
        pinecone_config=pinecone_config,
    )
    use_automatic_compute(update_pinecone_index)
    if optional_pipeline_input_provided(pinecone_connection_id):
        update_pinecone_index.environment_variables[
            "AZUREML_WORKSPACE_CONNECTION_ID_PINECONE"
        ] = pinecone_connection_id

    register_mlindex = register_mlindex_component(
        storage_uri=update_pinecone_index.outputs.index,
        asset_name=asset_name,
    )
    use_automatic_compute(register_mlindex)
    return {
        "mlindex_asset_uri": update_pinecone_index.outputs.index,
        "mlindex_asset_id": register_mlindex.outputs.asset_id,
    }

You can learn about the URIs AzureML will accept as data inputs [here](https://learn.microsoft.com/azure/machine-learning/how-to-read-write-data-v2?view=azureml-api-2&tabs=python#paths). Referencing a path on AzureML supported storages (Blob, ADLSgen2, ADLSgen1, Fileshare) works best using [Datastores](https://learn.microsoft.com/azure/machine-learning/how-to-datastore?view=azureml-api-2&tabs=cli-identity-based-access%2Ccli-adls-identity-based-access%2Ccli-azfiles-account-key%2Ccli-adlsgen1-identity-based-access) as they help manage credentials for access.

Here we will reference the Data Asset created from the DataImport Job.

In [ ]:
from azure.ai.ml import Input

input_data = Input(type="uri_folder", path=f"azureml:{import_asset_name}@latest")
data_source_url = "s3://my-bucket"

The `update_pinecone_index` component takes a `pinecone_config` argument which specifies the name of the Index to push chunked and embedded data to. If this index does not exist it will be created, if it does exists it will be reused.

**Note: if you are using Pinecone's Starter Plan, you are allowed to have only 1 index. In which case, make sure you have deleted any existing index if you are not looking to reuse.** More details on the Starter Plan here: https://docs.pinecone.io/docs/starter-environment.

In [ ]:
pinecone_config = {"index_name": "s3-data-import-aoai"}

print(update_pinecone_index_component.description)

Now we can create the Pipeline Job by calling the `@pipeline` annotated function and providing input arguments.
`asset_name` will be used when registering the MLIndex Data Asset produced by the `register_mlindex` component in the pipeline. This is how you can refer to the MLIndex within AzureML.

In [ ]:
index_asset_name = "s3_import_test"

In [ ]:
import json

pipeline_job = s3_into_pinecone(
    input_data=input_data,
    data_source_url=data_source_url,
    pinecone_config=json.dumps(pinecone_config),
    pinecone_connection_id=pinecone_connection_id,
    # Each run will save latest Embeddings to subfolder under this path, runs will load latest embeddings from container and reuse any unchanged chunk embeddings
    embeddings_container=Input(
        type="uri_folder",
        path=f"azureml://datastores/workspaceblobstore/paths/embeddings/{index_asset_name}",
    ),
    embeddings_model=embeddings_model_uri,
    # This should be None if using a HuggingFace embeddings model.
    aoai_connection_id=aoai_connection_id,
    # Name of asset to register MLIndex under
    asset_name=index_asset_name,
)

# By default AzureML Pipelines will reuse the output of previous component Runs when inputs have not changed.
# If you want to rerun the Pipeline every time each time so that any changes to upstream data sources are processed uncomment the below line.

# pipeline_job.settings.force_rerun = True

Finally we add some properties to `pipeline_job` which ensure the Index generation progress and final Artifact appear in the PromptFlow Vector Index UI.

In [ ]:
pipeline_job.properties["azureml.mlIndexAssetName"] = index_asset_name
pipeline_job.properties["azureml.mlIndexAssetKind"] = "pinecone"
pipeline_job.properties["azureml.mlIndexAssetSource"] = "Data Import - S3"

## Submit Pipeline

**In case of any errors see [TROUBLESHOOT.md](../../TROUBLESHOOT.md).**

The output of each step in the pipeline can be inspected via the Workspace UI, click the link under 'Details Page' after running the below cell. 

In [ ]:
running_pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="s3_to_pinecone"
)
running_pipeline_job

In [ ]:
ml_client.jobs.stream(running_pipeline_job.name)

## Use Index with langchain

The Data Asset produced by the AzureML Pipeline above contains a yaml file named 'MLIndex' which contains all the information needed to use the Azure Cognitive Search index.
For instance if an AOAI deployment was used to embed the documents the details of that deployment and a reference to the secret are there.
This allows easy loading of the MLIndex into a langchain retriever.

In [ ]:
from azureml.rag.mlindex import MLIndex

question = "What is RAG?"

retriever = MLIndex(
    ml_client.data.get(index_asset_name, label="latest")
).as_langchain_retriever()
retriever.get_relevant_documents(question)

If you have not deployed `gpt-35-turbo` on your Azure OpenAI resource the below cell will fail indicated the `API deployment for this resource does not exist`. Follow the previous instructions for deploying `text-embedding-ada-002` to deploy `gpt-35-turbo`, note the chosen deployment name below and use the same or update it if you choose different one.

In [ ]:
from langchain.chains import RetrievalQA
from azureml.rag.models import init_llm, parse_model_uri

model_config = parse_model_uri(
    "azure_open_ai://deployment/gpt-35-turbo/model/gpt-35-turbo"
)
model_config["api_base"] = aoai_connection["properties"]["target"]
model_config["key"] = aoai_connection["properties"]["credentials"]["key"]
model_config["temperature"] = 0.3
model_config["max_retries"] = 3

qa = RetrievalQA.from_chain_type(
    llm=init_llm(model_config), chain_type="stuff", retriever=retriever
)

qa.run(question)

Note: To control the number of documents returned when searching try getting the the MLIndex `as_langchain_vectorstore()` instead, this implements the `VectorStore` interface which has more parameters.

### [Optional] Provision Cluster

You don't have to! The settings on the Pipeline use AzureML Serverless Compute, you can use any SKU you have quota for on demand. If you want to use a cluster that's also supported.

In [ ]:
from azure.ai.ml.entities import AmlCompute

cpu_compute_target = "rag-cpu"

try:
    dedicated_cpu_compute = ml_client.compute.get(cpu_compute_target)
except Exception:
    # Let's create the Azure Machine Learning compute object with the intended parameters
    dedicated_cpu_compute = AmlCompute(
        name=cpu_compute_target,
        type="amlcompute",
        size="Standard_E8s_v3",
        min_instances=0,
        max_instances=2,
        idle_time_before_scale_down=600,
        tier="Dedicated",
    )

    dedicated_cpu_compute = ml_client.compute.begin_create_or_update(
        dedicated_cpu_compute
    ).result(timeout=600)